In [2]:
import polars as pl
import numpy as np
bio = pl.read_csv("lahman_people.csv")
pbp = pl.read_csv("savant_data_2021_2023.csv")
pbp

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,events,description,zone,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,…,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,role_key,sp_indicator,rp_indicator,pitch_number_appearance,pitcher_at_bat_number,times_faced
str,str,f64,f64,f64,str,str,str,str,i64,str,str,str,str,str,str,i64,str,i64,i64,i64,f64,f64,f64,f64,str,str,str,i64,i64,str,f64,f64,str,str,f64,f64,…,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,str,i64,i64,i64,i64,i64
"""FF""","""2021-04-05""",93.2,0.84,5.74,"""c7c83eaa9fe8da2f81c5fce172059a…","""4f902241478a103f7a818f5be9a7b7…","""strikeout""","""swinging_strike""",13,"""R""","""L""","""L""","""e58e9c0fd276f0c175345b6d952a9e…","""ae2caea4bb1f46b714e35f0176994f…","""S""",2,null,3,2,2021,0.43,1.57,-0.05,1.41,null,null,null,2,6,"""Bot""",null,null,"""55bd3201fa7465188c2c70169f2e94…",null,-3.263176,-135.5206,…,"""c04bfbc3f42ec36472468471b3a716…","""7694076619c298b4556297b20afaf8…","""17cbce1de7a9d4e31952e86ba69ced…","""b135b247f1afd053b201fc7c434493…","""fa9c99ce9acb5fd5971e8fb834d745…","""53dfe414229803162d14787d30fbf4…","""9b52235d7b694738421b6c2ed9271e…",53.47,null,null,0.0,1,0,0,null,47,7,"""4-Seam Fastball""",0,2,0,2,2,0,0,2,"""Infield shift""","""Standard""",166,-0.014,-0.134,"""SP""",1,0,97,22,3
"""SL""","""2021-04-05""",82.9,1.03,5.69,"""c7c83eaa9fe8da2f81c5fce172059a…","""4f902241478a103f7a818f5be9a7b7…",null,"""foul""",5,"""R""","""L""","""L""","""e58e9c0fd276f0c175345b6d952a9e…","""ae2caea4bb1f46b714e35f0176994f…","""S""",null,null,3,2,2021,-0.42,-0.34,0.09,2.24,null,null,null,2,6,"""Bot""",null,null,"""55bd3201fa7465188c2c70169f2e94…",null,-1.343797,-120.739091,…,"""c04bfbc3f42ec36472468471b3a716…","""7694076619c298b4556297b20afaf8…","""17cbce1de7a9d4e31952e86ba69ced…","""b135b247f1afd053b201fc7c434493…","""fa9c99ce9acb5fd5971e8fb834d745…","""53dfe414229803162d14787d30fbf4…","""9b52235d7b694738421b6c2ed9271e…",53.66,null,null,null,null,null,null,null,47,6,"""Slider""",0,2,0,2,2,0,0,2,"""Infield shift""","""Standard""",309,0.0,0.0,"""SP""",1,0,96,22,3
"""FF""","""2021-04-05""",94.6,-1.85,5.77,"""514eeb6e6c17085fc1adabf03f1adc…","""afb9b85defc6fe5c3f48681480eff4…","""caught_stealing_3b""","""ball""",14,"""R""","""R""","""R""","""3c4ec083bb4b200556718068a3cea7…","""284da2e9c8943a9faa155f56f0faaa…","""B""",2,null,2,2,2021,-0.56,1.53,2.01,1.92,null,"""c8f1ff43c331519aab03daab36b84b…",null,2,4,"""Top""",null,null,"""3103eeddc91e18c223c7ef4d3c12f2…",null,11.292119,-137.173707,…,"""a9b53974ac34cd7c03133051645ef4…","""c3b12c33078b6ce2f0c8287913cd68…","""88a241e142df6a547f9188eb81f5c2…","""c1efd3145c820257fae2347ff56758…","""4f6c643961e8df8f6d76d136b13ba2…","""a7ab335ba29a72bc194f77b694f23b…","""ccbeebb83e2f368d70aca48df9d3ca…",54.12,null,null,0.0,0,0,0,null,28,6,"""4-Seam Fastball""",0,4,4,0,4,0,4,0,"""Standard""","""Standard""",220,0.014,-0.202,"""SP""",1,0,95,19,3
"""FF""","""2021-04-05""",97.1,1.86,6.57,"""875eeca87c6f80182a88c2a7b92c04…","""0f061b9439845159c394a71e55d635…","""field_out""","""hit_into_play""",13,"""R""","""R""","""L""","""8499d67c7ea3ce9671a0574ff23364…","""febb17c5596d048b6a6a4eea23cfe0…","""X""",7,"""line_drive""",3,2,2021,0.75,1.56,-0.84,2.21,null,null,"""29df9634e71f12ec4754b370e647db…",2,5,"""Bot""",90.32,88.66,"""c4db34402ba417ae6cc0cf8eca1c05…",null,-8.928735,-140.780154,…,"""80f3a94c7bf1ed6397

In [143]:
def get_counts(data, type):#janky way to do this?
    """
    Gets count of playing time
    data - df of interest
    type - whether batter or pitcher
    """
    return (data
            .group_by(type,'game_pk', 'at_bat_number', 'game_year')
            .len().group_by(type, 'game_year').len()
            .rename({"len":"PA"}).pivot('game_year', index = type, values = 'PA'))
batterAB = pbp.pipe(get_counts, 'batter')
pitcherBF = pbp.pipe(get_counts, 'pitcher')
batterAB

batter,2023,2021,2022
str,u32,u32,u32
"""5b6b35fcf313a1e12256b4632795a0…",455,674,670
"""a540f5b1639f7c7d09108c6893fb30…",547,588,671
"""4380853322f8572ba35d3a2e8b25f8…",null,636,617
"""9dc0903fd6704325b19057c6733c91…",685,null,404
"""3ffca17a704b2bdd7b912cacb3c528…",null,5,null
…,…,…,…
"""6abe653fe11a79a725382f995ad6d2…",40,null,null
"""2d2f3ff492dab091a66dacfc0d50b4…",175,null,null
"""827135b2373b76725e3861585e8180…",null,20,null


In [122]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import root_mean_squared_error
#running linear regression of past 2 seasons to apply to 2023 - about as good as expected(pretty bad)
def test_model_sklearn(model, data, x, y):
    X_train, X_test, y_train, y_test = train_test_split(data.select(x), data.select(y), test_size = 0.25, random_state = 42) 
    #use random_state=42!!
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test) 
    return root_mean_squared_error(y_pred, y_test)
test_model_sklearn(LinearRegression(), batterAB.drop_nulls(), ['2022', '2021'], ['2023']), test_model_sklearn(LinearRegression(), pitcherBF.drop_nulls(), ['2022', '2021'], ['2023'])
#similar to averaging method - expected due to nature of MLR 
#baseline to beat: batter: 154.9507660967016, pitcher: 190.61133075412496

(154.9507660967016, 190.61133075412496)

In [124]:
def get_means(data):
    #using average of 2022, 2021 data to estimate 2023 data
    return (data.with_columns(pl.mean_horizontal(['2022', '2021'])
                                    .alias('row_mean')).drop_nulls(['2023', 'row_mean'])
                                    .select(['2023', 'row_mean']))
batter_test, batter_mean = batterAB.pipe(get_means)
pitcher_test, pitcher_mean = pitcherBF.pipe(get_means)
root_mean_squared_error(batter_test, batter_mean), root_mean_squared_error(pitcher_test, pitcher_mean)
#mean method: 178.68480531577185, 175.54575174805603 - about same as MLR method

(178.68480531577185, 175.54575174805603)

In [163]:
#need to predict nulls - looking at MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
def imputeData(estimator, batterdf, pitcherdf, max_iter = 200, random_state = 42):
    imputer = IterativeImputer(estimator = estimator, max_iter=max_iter, random_state=random_state)
    batterABimputed = (pl.DataFrame(imputer.fit_transform(
        batterdf.select("2023", "2022", "2021")), schema = ["2023", "2022", "2021"]).with_columns(
            batter = batterAB.get_column("batter")
    ))
    pitcherBFimputed = (pl.DataFrame(imputer.fit_transform(
        pitcherdf.select("2023", "2022", "2021")), schema = ["2023", "2022", "2021"]).with_columns(
            pitcher = pitcherBF.get_column("pitcher")
    ))
    return batterABimputed, pitcherBFimputed
batterimp, pitcherimp = imputeData(BayesianRidge(), batterAB, pitcherBF)#Bayesian Seems Best - actually converges
#batterimp, pitcherimp = imputeData(DecisionTreeRegressor(), batterAB, pitcherBF, max_iter = 2000)
#batterimp, pitcherimp = imputeData(KNeighborsRegressor(), batterAB, pitcherBF, max_iter = 2000)
test_model_sklearn(LinearRegression(), batterimp, ['2022', '2021'], ['2023']), test_model_sklearn(LinearRegression(), pitcherimp, ['2022', '2021'], ['2023'])
#MLR with imputation yields batter: 108.16645006899171, pitcher: 104.5259354514329


(108.16645006899171, 104.5259354514329)